# 使用NDArray来处理数据

对于机器学习来说，处理数据往往是万事之开头。它包含两个部分：(i)数据读取，(ii)数据已经在内存中时如何处理。本章将关注后者。

我们首先介绍NDArray，这是MXNet储存和变换数据的主要工具。如果你之前用过NumPy，你会发现NDArray和NumPy的多维数组非常类似。当然，NDArray提供更多的功能，首先是CPU和GPU的异步计算，其次是自动求导。这两点使得NDArray能更好地支持机器学习。

## 让我们开始

我们先介绍最基本的功能。如果你不懂我们用到的数学操作也不用担心，例如按元素加法、正态分布；我们会在之后的章节分别从数学和代码编写的角度详细介绍。

我们首先从mxnet导入ndarray这个包

In [1]:
from mxnet import ndarray as nd

然后我们创建一个3行和4列的2D数组（通常也叫**矩阵**），并且把每个元素初始化成0

In [2]:
nd.zeros((3, 4))


[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
<NDArray 3x4 @cpu(0)>

类似的，我们可以创建数组每个元素被初始化成1。

In [3]:
x = nd.ones((3, 4))
x


[[ 1.  1.  1.  1.]
 [ 1.  1.  1.  1.]
 [ 1.  1.  1.  1.]]
<NDArray 3x4 @cpu(0)>

或者从python的数组直接构造

In [4]:
nd.array([[1, 2], [2, 3]])


[[ 1.  2.]
 [ 2.  3.]]
<NDArray 2x2 @cpu(0)>

我们经常需要创建随机数组，即每个元素的值都是随机采样而来，这个经常被用来初始化模型参数。以下代码创建数组，它的元素服从均值0标准差1的正态分布。

In [6]:
y = nd.random_normal(0, 1, shape=(3, 4))
y


[[ 0.2444218  -0.03716067 -0.48774993 -0.02261727]
 [ 0.57461417  1.4661262   0.68629038  0.35496104]
 [ 1.07316959  0.12017461 -0.97111022 -0.77569664]]
<NDArray 3x4 @cpu(0)>

跟NumPy一样，每个数组的形状可以通过.shape来获取

In [7]:
y.shape

(3L, 4L)

In [8]:
y.size

12L

## 操作符

NDArray支持大量的数学操作符，例如按元素加法：

In [9]:
x + y


[[ 1.24442184  0.96283931  0.51225007  0.97738272]
 [ 1.57461417  2.4661262   1.68629038  1.35496104]
 [ 2.07316971  1.12017465  0.02888978  0.22430336]]
<NDArray 3x4 @cpu(0)>

乘法：

In [10]:
x * y


[[ 0.2444218  -0.03716067 -0.48774993 -0.02261727]
 [ 0.57461417  1.4661262   0.68629038  0.35496104]
 [ 1.07316959  0.12017461 -0.97111022 -0.77569664]]
<NDArray 3x4 @cpu(0)>

指数运算：

In [11]:
nd.exp(y)


[[ 1.27688277  0.9635213   0.6140064   0.97763658]
 [ 1.77644503  4.33241987  1.98633337  1.42612505]
 [ 2.9246347   1.12769377  0.37866241  0.46038294]]
<NDArray 3x4 @cpu(0)>

也可以转置一个矩阵然后计算矩阵乘法：

In [12]:
nd.dot(x, y.T)


[[-0.30310607  3.08199167 -0.55346262]
 [-0.30310607  3.08199167 -0.55346262]
 [-0.30310607  3.08199167 -0.55346262]]
<NDArray 3x3 @cpu(0)>

我们会在之后的线性代数一章讲解这些运算符。

## 广播(Broadcasting)

当二元操作符左右两边ndarray形状不一样时，系统会尝试将其复制到一个共同的形状。例如a的第0维是3，b的第0维是1，那么a+b时会将b沿着第0维复制3遍：

In [13]:
a = nd.arange(3).reshape((3, 1))
b = nd.arange(2).reshape((1, 2))
print('a:', a)
print('b:', b)
print('a+b:', a+b)

('a:', 
[[ 0.]
 [ 1.]
 [ 2.]]
<NDArray 3x1 @cpu(0)>)
('b:', 
[[ 0.  1.]]
<NDArray 1x2 @cpu(0)>)
('a+b:', 
[[ 0.  1.]
 [ 1.  2.]
 [ 2.  3.]]
<NDArray 3x2 @cpu(0)>)


## 跟NumPy的转换

ndarray可以很方便地同numpy进行转换

In [14]:
import numpy as np
x = np.ones((2, 3))
y = nd.array(x) # numpy -> mxnet
z = y.asnumpy() # mxnet -> numpy
print([z, y])

[array([[ 1.,  1.,  1.],
       [ 1.,  1.,  1.]], dtype=float32), 
[[ 1.  1.  1.]
 [ 1.  1.  1.]]
<NDArray 2x3 @cpu(0)>]


## 替换操作

在前面的样例中，我们为每个操作新开内存来存储它的结果。例如，如果我们写y = x + y，我们会把y从现在指向的实例转到新建的实例上来。我们可以用Python的id()函数来看这个是怎么执行的：

In [15]:
x = nd.ones((3, 4))
y = nd.ones((3, 4))

before = id(y)
y = y + x
id(y) == before

False

我们可以把结果通过\[:\]写到一个之前开好的数组里：

In [16]:
z = nd.zeros_like(x)
before = id(z)
z[:] = x + y
id(z) == before

True

但是这里我们还是为x+y创建了临时空间，然后再复制到z。需要避免这个开销，我们可以使用操作符的全名版本中的out参数：

In [17]:
nd.elemwise_add(x, y, out=z)
id(z) == before

True

如果现有的数组不会复用，我们也可以用x\[:\] = x + y，或者x += y达到这个目的：

In [18]:
before = id(x)
x += y
id(x) == before

True

## 截取(Slicing)

MXNet NDArray提供了各种截取方法。截取x的index的1、2的列：

In [19]:
x = nd.arange(0, 9).reshape((3, 3))
print('x: ', x)
x[1:3]

('x: ', 
[[ 0.  1.  2.]
 [ 3.  4.  5.]
 [ 6.  7.  8.]]
<NDArray 3x3 @cpu(0)>)



[[ 3.  4.  5.]
 [ 6.  7.  8.]]
<NDArray 2x3 @cpu(0)>

以及直接写入指定位置：

In [20]:
x[1, 2] = 9.0
x


[[ 0.  1.  2.]
 [ 3.  4.  9.]
 [ 6.  7.  8.]]
<NDArray 3x3 @cpu(0)>

多维截取：

In [21]:
x = nd.arange(0, 9).reshape((3, 3))
print('x: ', x)
x[1:2, 1:3]

('x: ', 
[[ 0.  1.  2.]
 [ 3.  4.  5.]
 [ 6.  7.  8.]]
<NDArray 3x3 @cpu(0)>)



[[ 4.  5.]]
<NDArray 1x2 @cpu(0)>

多维写入：

In [22]:
x[1:2, 1:3] = 9.0
x


[[ 0.  1.  2.]
 [ 3.  9.  9.]
 [ 6.  7.  8.]]
<NDArray 3x3 @cpu(0)>

## 总结

ndarray模块提供一系列多维数组操作函数。所有函数列表可以参见[NDArray API文档](https://mxnet.incubator.apache.org/api/python/ndarray.html)。